In [123]:
%logstop
%logstart -rtq ~/.logs/ml.py append
import seaborn as sns
sns.set()

In [2]:
from static_grader import grader

# ML Miniproject
## Introduction

The objective of this miniproject is to exercise your ability to create effective machine learning models for making predictions. We will be working with nursing home inspection data from the United States, predicting which providers may be fined and for how much.

## Scoring

In this miniproject you will often submit your model's `predict` or `predict_proba` method to the grader. The grader will assess the performance of your model using a scoring metric, comparing it against the score of a reference model. We will use the [average precision score](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.average_precision_score.html). If your model performs better than the reference solution, then you can score higher than 1.0.

**Note:** If you use an estimator that relies on random draws (like a `RandomForestClassifier`) you should set the `random_state=` to an integer so that your results are reproducible. 

## Downloading the data

We can download the data set from Amazon S3:

In [3]:
%%bash
mkdir data
wget http://dataincubator-wqu.s3.amazonaws.com/mldata/providers-train.csv -nc -P ./ml-data
wget http://dataincubator-wqu.s3.amazonaws.com/mldata/providers-metadata.csv -nc -P ./ml-data

mkdir: cannot create directory ‘data’: File exists
File ‘./ml-data/providers-train.csv’ already there; not retrieving.

File ‘./ml-data/providers-metadata.csv’ already there; not retrieving.



We'll load the data into a Pandas DataFrame. Several columns will become target labels in future questions. Let's pop those columns out from the data, and drop related columns that are neither targets nor reasonable features (i.e. we don't wouldn't know how many times a facility denied payment before knowing whether it was fined).

The data has many columns. We have also provided a data dictionary.

In [4]:
import numpy as np
import pandas as pd

In [5]:
metadata = pd.read_csv('./ml-data/providers-metadata.csv')
metadata.head(40)

,Variable,Label,Description,Format
0,PROVNUM,Federal Provider Number,Federal Provider Number,6 alphanumeric characters
1,PROVNAME,Provider Name,Provider Name,text
2,ADDRESS,Provider Address,Provider Address,text
3,CITY,Provider City,Provider City,text
4,STATE,Provider State,Provider State,2-character postal abbreviation
5,ZIP,Provider Zip Code,Provider Zip Code,5-digit zip code
6,PHONE,Provider Phone Number,Provider Phone Number,10 digit phone number
7,COUNTY_SSA,Provider SSA County,SSA county code,3-digit SSA code
8,COUNTY_NAME,Provider County Name,Provider County Name,text
9,OWNERSHIP,Ownership Type,Nature of organization that operates a provide...,text


In [6]:
metadata.tail(40)

,Variable,Label,Description,Format
39,PTHRD,Reported Physical Therapist Staffing Hours per...,Reported Physical Therapy Staffing - Hours per...,"real number, up to 5 decimal places"
40,exp_aide,Expected Nurse Aide Staffing Hours per Residen...,Expected Nurse Aide Staffing - Hours per Resid...,"real number, up to 5 decimal places"
41,exp_lpn,Expected LPN Staffing Hours per Resident per Day,Expected LPN Staffing - Hours per Resident per...,"real number, up to 5 decimal places"
42,exp_rn,Expected RN Staffing Hours per Resident per Day,Expected RN Staffing - Hours per Resident per Day,"real number, up to 5 decimal places"
43,exp_total,Expected Total Nurse Staffing Hours per Reside...,Expected Total Nurse Staffing - Hours per Resi...,"real number, up to 5 decimal places"
44,adj_aide,Adjusted Nurse Aide Staffing Hours per Residen...,Adjusted Nurse Aide Staffing - Hours per Resid...,"real number, up to 5 decimal places"
45,adj_lpn,Adjusted LPN Staffing Hours per Resident per Day,Adjusted LPN Staffing - Hours per Resident per...,"real number, up to 5 decimal places"
46,adj_rn,Adjusted RN Staffing Hours per Resident per Day,Adjusted RN Staffing - Hours per Resident per Day,"real number, up to 5 decimal places"
47,adj_total,Adjusted Total Nurse Staffing Hours per Reside...,Adjusted Total Nurse Staffing - Hours per Resi...,"real number, up to 5 decimal places"
48,R_SRVY_DT_F,Health Survey Date Under New Process,Date of most recent health inspection occurrin...,date


In [7]:
data = pd.read_csv('./ml-data/providers-train.csv', encoding='latin1')

fine_counts = data.pop('FINE_CNT')
fine_totals = data.pop('FINE_TOT')
cycle_2_score = data.pop('CYCLE_2_TOTAL_SCORE')

In [8]:
data.head()

,PROVNUM,PROVNAME,ADDRESS,CITY,STATE,ZIP,PHONE,COUNTY_SSA,COUNTY_NAME,BEDCERT,...,CERTIFICATION,CYCLE_1_DEFS,CYCLE_1_NFROMDEFS,CYCLE_1_NFROMCOMP,CYCLE_1_DEFS_SCORE,CYCLE_1_NUMREVIS,CYCLE_1_REVISIT_SCORE,CYCLE_1_TOTAL_SCORE,CYCLE_1_SURVEY_DATE,CYCLE_2_SURVEY_DATE
0,015010,COOSA VALLEY NURSING FACILITY,315 WEST HICKORY STREET,SYLACAUGA,AL,35150,2562495604,600,Talladega,85,...,Medicare and Medicaid,7,7,0,36,1,0,36,2017-04-06,2016-05-26
1,015012,HIGHLANDS HEALTH AND REHAB,380 WOODS COVE ROAD,SCOTTSBORO,AL,35768,2562183708,350,Jackson,50,...,Medicare and Medicaid,5,5,0,44,1,0,44,2017-03-16,2016-02-04
2,015014,EASTVIEW REHABILITATION & HEALTHCARE CENTER,7755 FOURTH AVENUE SOUTH,BIRMINGHAM,AL,35206,2058330146,360,Jefferson,92,...,Medicare and Medicaid,6,6,0,40,1,0,40,2016-10-20,2015-12-30
3,015015,PLANTATION MANOR NURSING HOME,6450 OLD TUSCALOOSA HIGHWAY P O BOX 97,MC CALLA,AL,35111,2054776161,360,Jefferson,103,...,Medicare and Medicaid,2,2,0,16,1,0,16,2017-03-09,2016-02-11
4,015016,ATHENS HEALTH AND REHABILITATION LLC,611 WEST MARKET STREET,ATHENS,AL,35611,2562321620,410,Limestone,149,...,Medicare and Medicaid,2,2,0,20,1,0,20,2017-06-01,2016-05-12


In [9]:
data.columns

Index(['PROVNUM', 'PROVNAME', 'ADDRESS', 'CITY', 'STATE', 'ZIP', 'PHONE',
       'COUNTY_SSA', 'COUNTY_NAME', 'BEDCERT', 'RESTOT', 'INHOSP',
       'CCRC_FACIL', 'SFF', 'CHOW_LAST_12MOS', 'SPRINKLER_STATUS', 'EXP_TOTAL',
       'ADJ_TOTAL', 'OWNERSHIP', 'CERTIFICATION', 'CYCLE_1_DEFS',
       'CYCLE_1_NFROMDEFS', 'CYCLE_1_NFROMCOMP', 'CYCLE_1_DEFS_SCORE',
       'CYCLE_1_NUMREVIS', 'CYCLE_1_REVISIT_SCORE', 'CYCLE_1_TOTAL_SCORE',
       'CYCLE_1_SURVEY_DATE', 'CYCLE_2_SURVEY_DATE'],
      dtype='object')

In [10]:
dict(data.groupby('STATE')['CYCLE_1_TOTAL_SCORE'].mean())

{'AK': 90.9375,
 'AL': 41.6747572815534,
 'AR': 87.82089552238806,
 'AZ': 23.634920634920636,
 'CA': 87.62033582089552,
 'CO': 72.36224489795919,
 'CT': 44.577669902912625,
 'DC': 81.53333333333333,
 'DE': 63.71052631578947,
 'FL': 41.36260162601626,
 'GA': 42.7875,
 'GU': 200.0,
 'HI': 67.28571428571429,
 'IA': 37.60459183673469,
 'ID': 137.84057971014494,
 'IL': 67.47457627118644,
 'IN': 52.40495867768595,
 'KS': 117.47260273972603,
 'KY': 47.78076923076923,
 'LA': 31.951807228915662,
 'MA': 45.00854700854701,
 'MD': 86.5273631840796,
 'ME': 19.445652173913043,
 'MI': 105.79746835443038,
 'MN': 40.97959183673469,
 'MO': 47.91609977324263,
 'MS': 28.128342245989305,
 'MT': 99.47058823529412,
 'NC': 57.96962025316456,
 'ND': 39.828947368421055,
 'NE': 50.202127659574465,
 'NH': 13.735294117647058,
 'NJ': 23.869300911854104,
 'NM': 160.08823529411765,
 'NV': 40.589285714285715,
 'NY': 31.436265709156196,
 'OH': 45.99531066822978,
 'OK': 95.54074074074074,
 'OR': 78.45901639344262,
 'PA'

In [11]:
# for state in 

## Question 1: state_model

A federal agency, Centers for Medicare and Medicaid Services (CMS), imposes regulations on nursing homes. However, nursing homes are inspected by state agencies for compliance with regulations, and fines for violations can vary widely between states.

Let's develop a very simple initial model to predict the amount of fines a nursing home might expect to pay based on its location. Fill in the class definition of the custom estimator, `StateMeanEstimator`, below.

**Note:** When the grader checks your answer, it passes a list of dictionaries to the `predict` method of your estimator, not a DataFrame. This means that your model must work with both data types. You can handle this by adding a test (and optional conversion) in the `predict` method of your custom class, similar to the `ColumnSelectTransformer` given below in Question 2.  

In [12]:
from sklearn.base import BaseEstimator, RegressorMixin, TransformerMixin

class GroupMeanEstimator(BaseEstimator, RegressorMixin):
    def __init__(self, grouper):
        self.grouper = grouper
        self.group_averages = {}

    def fit(self, X, y):
        # Use self.group_averages to store the average penalty by group
        Xt=X.copy()
        Xt['FINES']=y
        self.group_averages_=dict(Xt.groupby(self.grouper)['FINES'].mean())
        self.y_mean_=y.mean()
        return self

    def predict(self, X):
        # Return a list of predicted penalties based on group of samples in X
        Xt=pd.DataFrame(X)
        for state in Xt[self.grouper]:
            if state not in self.group_averages_.keys():
                self.group_averages_[state]=self.y_mean_
        return [self.group_averages_[x] for x in Xt[self.grouper]]

After filling in class definition, we can create an instance of the estimator and fit it to the data.

In [13]:
from sklearn.pipeline import Pipeline

state_model = Pipeline([
    ('sme', GroupMeanEstimator(grouper='STATE'))
    ])
state_model.fit(data, fine_totals)

Pipeline(steps=[('sme', GroupMeanEstimator(grouper='STATE'))])

Next we should test that our predict method works.

In [14]:
state_model.predict(data.sample(5))

[32656.315384615384,
 18147.119744058502,
 8214.822977725675,
 18147.119744058502,
 12365.983606557376]

However, what if we have data from a nursing home in a state (or territory) of the US which is not in the training data?

In [15]:
state_model.predict(pd.DataFrame([{'STATE': 'AS'}]))

[14969.857687877915]

Make sure your model can handle this possibility before submitting your model's predict method to the grader.

In [18]:
grader.score.ml__state_model(state_model.predict)

Your score: 1.000


## Question 2: simple_features_model

Nursing homes vary greatly in their business characteristics. Some are owned by the government or non-profits while others are run for profit. Some house a few dozen residents while others house hundreds. Some are located within hospitals and may work with more vulnerable populations. We will try to predict which facilities are fined based on their business characteristics.

We'll begin with columns in our DataFrame containing numeric and boolean features. Some of the rows contain null values; estimators cannot handle null values so these must be imputed or dropped. We will create a `Pipeline` containing transformers that process these features, followed by an estimator.

**Note:** As mentioned above in Question 1, when the grader checks your answer, it passes a list of dictionaries to the `predict` or `predict_proba` method of your estimator, not a DataFrame. This means that your model must work with both data types. For this reason, we've provided a custom `ColumnSelectTransformer` for you to use instead `scikit-learn`'s own `ColumnTransformer`.

In [76]:

simple_cols = ['BEDCERT', 'RESTOT', 'INHOSP', 'CCRC_FACIL', 'SFF', 'CHOW_LAST_12MOS', 'SPRINKLER_STATUS', 'EXP_TOTAL', 'ADJ_TOTAL']

class ColumnSelectTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)
          
        return X[self.columns]
        
simple_features = Pipeline([
    ('cst', ColumnSelectTransformer(simple_cols)),
])

In [70]:
# from sklearn.impute import SimpleImputer
# my_imputer=SimpleImputer(strategy='mean')
# imp_x=pd.DataFrame(my_imputer.fit_transform(data[simple_cols]))
# imp_x.columns=simple_cols

In [81]:
SimpleImputer?

In [82]:
from sklearn.impute import SimpleImputer
imr = SimpleImputer( strategy='mean')
fit_imr = imr.fit(data[simple_cols])
imputed_values = fit_imr.transform(data[simple_cols])

idata = pd.DataFrame(imputed_values, columns=simple_cols)

my_pipeline = Pipeline([
    ('imput', imr)
])

simple_features = Pipeline([ ('cst', ColumnSelectTransformer(simple_cols)), ('imput', imr), ])

In [71]:
# simple_features=Pipeline([
#     ('cst', ColumnSelectTransformer(simple_cols)),('imputer',my_imputer),
# ])

**Note:** The assertion below assumes the output of `noncategorical_features.fit_transform` is a `ndarray`, not a `DataFrame`.)

In [83]:
assert data['RESTOT'].isnull().sum() > 0
assert not np.isnan(simple_features.fit_transform(data)).any()

Now combine the `simple_features` pipeline with an estimator in a new pipeline. Fit `simple_features_model` to the data and submit `simple_features_model.predict_proba` to the grader. You may wish to use cross-validation to tune the hyperparameters of your model.

In [84]:
from sklearn import linear_model,decomposition
pca = decomposition.PCA()

simple_features_model = Pipeline([
    ('simple', simple_features),
    ('rgr',linear_model.LogisticRegression())
    # add your estimator here
])

In [85]:
simple_features_model.fit(data, fine_counts > 0)

/opt/conda/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('simple',
                 Pipeline(steps=[('cst',
                                  ColumnSelectTransformer(columns=['BEDCERT',
                                                                   'RESTOT',
                                                                   'INHOSP',
                                                                   'CCRC_FACIL',
                                                                   'SFF',
                                                                   'CHOW_LAST_12MOS',
                                                                   'SPRINKLER_STATUS',
                                                                   'EXP_TOTAL',
                                                                   'ADJ_TOTAL'])),
                                 ('imput', SimpleImputer())])),
                ('rgr', LogisticRegression())])

In [86]:
def positive_probability(model):
    def predict_proba(X):
        return model.predict_proba(X)[:, 1]
    return predict_proba

grader.score.ml__simple_features(positive_probability(simple_features_model))

Your score: 0.996


## Question 3: categorical_features

The `'OWNERSHIP'` and `'CERTIFICATION'` columns contain categorical data. We will have to encode the categorical data into numerical features before we pass them to an estimator. Construct one or more pipelines for this purpose. Transformers such as [LabelEncoder](https://scikit-learn.org/0.19/modules/generated/sklearn.preprocessing.LabelEncoder.html#sklearn.preprocessing.LabelEncoder) and [OneHotEncoder](https://scikit-learn.org/0.19/modules/generated/sklearn.preprocessing.OneHotEncoder.html#sklearn.preprocessing.OneHotEncoder) may be useful, but you may also want to define your own transformers.

If you used more than one `Pipeline`, combine them with a `FeatureUnion`. As in Question 2, we will combine this with an estimator, fit it, and submit the `predict_proba` method to the grader.

In [24]:
data.OWNERSHIP.unique()

array(['For profit - Corporation', 'Government - County',
       'Non profit - Other', 'Non profit - Corporation',
       'For profit - Individual', 'For profit - Partnership',
       'Government - City', 'Non profit - Church related',
       'Government - City/county', 'Government - Federal',
       'For profit - Limited Liability company', 'Government - State',
       'Government - Hospital district'], dtype=object)

In [25]:
data.CERTIFICATION.unique()

array(['Medicare and Medicaid', 'Medicare', 'Medicaid'], dtype=object)

In [87]:
from sklearn.preprocessing import OneHotEncoder
class One_hot(BaseEstimator, TransformerMixin):
    def __init__(self,column):
        self.column=column
        
    def fit(self,X,y=None):
        return self
    
    def transform(self,X):
        if not isinstance(X,pd.DataFrame):
            X=pd.DataFrame(X)
            
        X=pd.get_dummies(X,drop_first=True)
        return X
        
    
        

In [29]:
pd.get_dummies?

In [88]:
from sklearn.pipeline import FeatureUnion

owner_onehot = Pipeline([
    ('cst', ColumnSelectTransformer(['OWNERSHIP'])),
    ('onehot',One_hot('OWNERSHIP')),
])

cert_onehot = Pipeline([
    ('cst', ColumnSelectTransformer(['CERTIFICATION'])),
    ('onehot',One_hot('CERTIFICATION')),
])

categorical_features = FeatureUnion([('cst-own',owner_onehot),('cst-cert',cert_onehot),
])

In [89]:
assert categorical_features.fit_transform(data).shape[0] == data.shape[0]
assert categorical_features.fit_transform(data).dtype == np.float64
assert not np.isnan(categorical_features.fit_transform(data)).any()

AssertionError: 

As in the previous question, create a model using the `categorical_features`, fit it to the data, and submit its `predict_proba` method to the grader.

In [55]:
# utils.class_weight?

Object `utils.class_weight` not found.


In [56]:
from sklearn import model_selection,utils

class_weights=utils.class_weight.compute_class_weight('balanced',np.unique(fine_counts > 0),fine_counts > 0)

classes = np.unique(fine_counts>0)

class_weights = dict(zip(classes,class_weights))

/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass classes=[False  True], y=0         True
1        False
2        False
3        False
4        False
         ...  
13887    False
13888    False
13889    False
13890    False
13891     True
Name: FINE_CNT, Length: 13892, dtype: bool as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


In [57]:
categorical_features_model = Pipeline([
    ('categorical', categorical_features),
    ('estimator',tree.DecisionTreeClassifier(random_state=42, max_depth=5, class_weight=class_weights))
    # add your estimator here
])

In [58]:
categorical_features_model.fit(data, fine_counts > 0)

Pipeline(steps=[('categorical',
                 FeatureUnion(transformer_list=[('cst-own',
                                                 Pipeline(steps=[('cst',
                                                                  ColumnSelectTransformer(columns=['OWNERSHIP'])),
                                                                 ('onehot',
                                                                  One_hot(column='OWNERSHIP'))])),
                                                ('cst-cert',
                                                 Pipeline(steps=[('cst',
                                                                  ColumnSelectTransformer(columns=['CERTIFICATION'])),
                                                                 ('onehot',
                                                                  One_hot(column='CERTIFICATION'))]))])),
                ('estimator',
                 DecisionTreeClassifier(class_weight={False: 0.7226383687057845,

In [59]:
grader.score.ml__categorical_features(positive_probability(categorical_features_model))

Your score: 1.058


## Question 4: business_model

Finally, we'll combine `simple_features` and `categorical_features` in a `FeatureUnion`, followed by an estimator in a `Pipeline`. You may want to optimize the hyperparameters of your estimator using cross-validation or try engineering new features (e.g. see [PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html)). When you've assembled and trained your model, pass the `predict_proba` method to the grader.

In [64]:
business_features = FeatureUnion([
    ('simple', simple_features),
    ('categorical', categorical_features)
])

In [66]:
from sklearn import tree
from sklearn.preprocessing import PolynomialFeatures
business_model = Pipeline([
    ('features', business_features),
    ('poly',PolynomialFeatures(2)),
    ('lr', linear_model.LogisticRegression())
    # add your estimator here
])

In [42]:
# tree.DecisionTreeClassifier?

In [43]:
# from sklearn.model_selection import GridSearchCV
# model=tree.DecisionTreeClassifier()
# gs=GridSearchCV(model,
#                {
#                    'max_depth':range(1,15),
#                    'min_samples_split': range(2,50,2),
#                },
#                cv=5,
#                n_jobs=2,
#                verbose=1)
# gs.fit(data,fine_counts)

In [67]:
business_model.fit(data, fine_counts > 0)

/opt/conda/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('simple',
                                                 Pipeline(steps=[('cst',
                                                                  ColumnSelectTransformer(columns=['BEDCERT',
                                                                                                   'RESTOT',
                                                                                                   'INHOSP',
                                                                                                   'CCRC_FACIL',
                                                                                                   'SFF',
                                                                                                   'CHOW_LAST_12MOS',
                                                                                                   'SPRINKLER_STATUS',
                                                       

In [68]:
grader.score.ml__business_model(positive_probability(business_model))

Your score: 0.760


## Question 5: survey_results

Surveys reveal safety and health deficiencies at nursing homes that may indicate risk for incidents (and penalties). CMS routinely makes surveys of nursing homes. Build a model that combines the `business_features` of each facility with its cycle 1 survey results, as well as the time between the cycle 1 and cycle 2 survey to predict the cycle 2 total score.

First, let's create a transformer to calculate the difference in time between the cycle 1 and cycle 2 surveys.

In [107]:
pd.DataFrame(pd.to_datetime(data.CYCLE_1_SURVEY_DATE)-pd.to_datetime(data.CYCLE_2_SURVEY_DATE))

,0
0,315 days
1,406 days
2,295 days
3,392 days
4,385 days
...,...
13887,309 days
13888,336 days
13889,288 days
13890,331 days


In [106]:
pd.DataFrame?

In [131]:
class TimedeltaTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, t1_col, t2_col):
        self.t1_col = t1_col
        self.t2_col = t2_col

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)
        Xr=X.copy()
        Xr['date1'] = pd.to_datetime(Xr[self.t1_col]) 
        Xr['date2'] = pd.to_datetime(Xr[self.t2_col])
        
        return Xr[['date1','date2']].apply(lambda x: (x[0] - x[1]).days, axis=1).to_frame(name='TIMEDELTA')

In [132]:
cycle_1_date = 'CYCLE_1_SURVEY_DATE'
cycle_2_date = 'CYCLE_2_SURVEY_DATE'
time_feature = TimedeltaTransformer(cycle_1_date, cycle_2_date)

In the cell below we'll collect the cycle 1 survey features.

In [133]:
cycle_1_cols = ['CYCLE_1_DEFS', 'CYCLE_1_NFROMDEFS', 'CYCLE_1_NFROMCOMP',
                'CYCLE_1_DEFS_SCORE', 'CYCLE_1_NUMREVIS',
                'CYCLE_1_REVISIT_SCORE', 'CYCLE_1_TOTAL_SCORE']
cycle_1_features = ColumnSelectTransformer(cycle_1_cols)

In [136]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import TruncatedSVD
survey_model = Pipeline([
    ('features', FeatureUnion([
        ('business', business_features),
        ('survey', cycle_1_features),
        ('time', time_feature)
    ])),
    ("poly",PolynomialFeatures(2)),
    ("svd",TruncatedSVD(20) ),
    ('reg',RandomForestRegressor(random_state = 0,n_estimators=200, min_samples_leaf = 100))# add your estimator here
])

In [138]:
survey_model.fit(data, cycle_2_score.astype(int))

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('business',
                                                 FeatureUnion(transformer_list=[('simple',
                                                                                 Pipeline(steps=[('cst',
                                                                                                  ColumnSelectTransformer(columns=['BEDCERT',
                                                                                                                                   'RESTOT',
                                                                                                                                   'INHOSP',
                                                                                                                                   'CCRC_FACIL',
                                                                                                                                   'SFF',
       

In [139]:
grader.score.ml__survey_model(survey_model.predict)

Your score: 1.240


*Copyright &copy; 2021 WorldQuant University. This content is licensed solely for personal use. Redistribution or publication of this material is strictly prohibited.*